In [59]:
import pandas as pd
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import string

In [64]:
data = pd.read_csv(r"E:\EEG DATA\train_ordered\02_tcp_le\Text_data\text_meta.csv")
data.head()
data['edf_location'][0]

'E:/EEG DATA/train_ordered/02_tcp_le/Formatted/00000002/00000002_s005.txt'

In [65]:
import sys
#Function to retrieve the medical reports
def get_report(link):
    try:
        with open(link, 'r') as myfile:
            txt = myfile.read().replace('\n', '')
    except:
        print("Encoding Error")
        return "None"
    myfile.close()
    return txt

#data['edf_location'] = data['edf_location'].progress_apply(lambda txt: str(txt))
data['report'] = data['edf_location'].progress_apply(lambda link: get_report(link))

Encoding Error
Encoding Error


In [66]:
#Just two files caused a great deal of trouble!

In [67]:
print(data.head())
data['report'] = data['report'].apply(lambda txt: str(txt))

                                        edf_location  label  \
0  E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...      0   
1  E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...      1   
2  E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...      0   
3  E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...      0   
4  E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...      0   

                                              report  
0  CLINICAL HISTORY: This is a 55-year-old, right...  
1  CLINICAL HISTORY: This is a 27-year-old woman ...  
2  CLINICAL HISTORY: This is a 23-year-old male w...  
3  CLINICAL HISTORY: 56 year old male with a smal...  
4  CLINICAL HISTORY: 56 year old male with a smal...  


In [68]:
#Word Tokenization
stop_words=set(stopwords.words("english"))
print("Stopwords from the NLTK corpus: ", stop_words)

def text_process(text):
    #String punctuation provides all the necessary checks
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

data['report'] = data['report'].progress_apply(lambda txt: text_process(txt))

Stopwords from the NLTK corpus:  {'have', 'such', 'our', 'the', 'into', 'until', "isn't", 'herself', 'out', 'has', 'only', 'aren', 'some', 'nor', 'just', "doesn't", 'does', 'during', "you'd", 'itself', 'be', 'him', 'to', "hasn't", 'her', 'over', 'any', 'didn', 'mustn', "aren't", 'don', 'where', 'theirs', 'as', 'under', 'is', 'off', "you'll", 'more', 'wasn', 'o', 'himself', 'because', 'these', 'll', 'other', 'was', 'about', 'how', 'ours', 'both', "shouldn't", 'hadn', 'won', 'ma', "it's", 'further', 'couldn', 'did', 'had', 'at', 'than', 'again', 'a', "weren't", 'can', 'do', 'all', 'themselves', 'wouldn', 'having', 'shouldn', 'each', 'been', 'few', 'what', 've', "should've", 'it', 't', 'whom', 'too', 'that', 'being', 'while', 'i', "that'll", "hadn't", 'yourselves', 'most', 'from', "wouldn't", 'and', 'we', 'y', 'she', 'ain', 'above', 're', 'with', 'needn', "needn't", "you've", 'when', 'shan', 'up', 'd', 'haven', 'for', "haven't", 'you', 'so', 'then', 'below', 'my', 'of', 'or', 'down', 'wer

In [80]:
def join_back(txt):
    new = ''
    for x in txt:
        new += ''.join(x)+" "
    return new

data['report'] = data['report'].progress_apply(lambda txt: join_back(txt))

In [82]:
data.head()

,edf_location,label,report
0,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,0,CLINICAL HISTORY 55yearold righthanded woman C...
1,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,1,CLINICAL HISTORY 27yearold woman focal scleros...
2,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,0,CLINICAL HISTORY 23yearold male history seizur...
3,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,0,CLINICAL HISTORY 56 year old male small lacuna...
4,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,0,CLINICAL HISTORY 56 year old male small lacuna...


In [83]:
data.to_csv(r"E:\EEG DATA\train_ordered\02_tcp_le\Text_data\text_meta.csv")

In [85]:
#Document Term Matrix
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
#Tokenizing and Cleaning - again - satisfaction
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (2,2),tokenizer = token.tokenize)
text_counts = cv.fit_transform(data['report'])

In [87]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text_counts, data['label'], test_size=0.2, random_state=1)

In [93]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

embd_size=32
vocab_size = text_counts.shape[1]

model=Sequential()
model.add(Embedding(vocab_size, embd_size, input_length = text_counts.shape[1]))
model.add(LSTM(200))
#Add dropout - Regularization
#Add ensemble of engineered features
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 17362, 32)         555584    
_________________________________________________________________
lstm_3 (LSTM)                (None, 200)               186400    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 201       
Total params: 742,185
Trainable params: 742,185
Non-trainable params: 0
_________________________________________________________________
None


In [94]:
model.fit(X_train, y_train, epochs=5, batch_size=5)

Epoch 1/5
144/144 [==============================] - ETA: 3:19 - loss: 0.6931 - acc: 0.800 - ETA: 3:13 - loss: 0.6958 - acc: 0.600 - ETA: 3:11 - loss: 0.6930 - acc: 0.600 - ETA: 3:06 - loss: 0.6953 - acc: 0.550 - ETA: 3:01 - loss: 0.7000 - acc: 0.480 - ETA: 2:55 - loss: 0.6956 - acc: 0.533 - ETA: 2:48 - loss: 0.6925 - acc: 0.571 - ETA: 2:41 - loss: 0.6935 - acc: 0.550 - ETA: 2:35 - loss: 0.6927 - acc: 0.555 - ETA: 2:30 - loss: 0.6920 - acc: 0.560 - ETA: 2:26 - loss: 0.6947 - acc: 0.527 - ETA: 2:22 - loss: 0.6939 - acc: 0.533 - ETA: 2:18 - loss: 0.6919 - acc: 0.553 - ETA: 2:14 - loss: 0.6927 - acc: 0.542 - ETA: 2:10 - loss: 0.6934 - acc: 0.533 - ETA: 2:08 - loss: 0.6929 - acc: 0.537 - ETA: 2:03 - loss: 0.6924 - acc: 0.541 - ETA: 1:56 - loss: 0.6908 - acc: 0.555 - ETA: 1:48 - loss: 0.6892 - acc: 0.568 - ETA: 1:40 - loss: 0.6863 - acc: 0.590 - ETA: 1:31 - loss: 0.6846 - acc: 0.600 - ETA: 1:22 - loss: 0.6826 - acc: 0.609 - ETA: 1:12 - loss: 0.6823 - acc: 0.608 - ETA: 1:04 - loss: 0.6793 - 

In [95]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 64.86%
